In [39]:
import pandas as pd
import numpy as np
import seaborn as sns

import pylab
import matplotlib.pyplot as plt
from scipy import stats
from feature_engine.outliers import Winsorizer



from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split



dia=pd.read_csv("Diabetes.csv")

dia.info()

dia.columns=["Pregnancies","Glucose","BP","Skin","Insulin","BMI","Pedigree","Age","Outcome"]


<class 'pandas.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0    Number of times pregnant      768 non-null    int64  
 1    Plasma glucose concentration  768 non-null    int64  
 2    Diastolic blood pressure      768 non-null    int64  
 3    Triceps skin fold thickness   768 non-null    int64  
 4    2-Hour serum insulin          768 non-null    int64  
 5    Body mass index               768 non-null    float64
 6    Diabetes pedigree function    768 non-null    float64
 7    Age (years)                   768 non-null    int64  
 8    Class variable                768 non-null    str    
dtypes: float64(2), int64(6), str(1)
memory usage: 55.9 KB


In [41]:
cols=dia.columns

print(dia.isna().sum())
print(dia.isnull().any())

print(dia.describe())

Pregnancies    0
Glucose        0
BP             0
Skin           0
Insulin        0
BMI            0
Pedigree       0
Age            0
Outcome        0
dtype: int64
Pregnancies    False
Glucose        False
BP             False
Skin           False
Insulin        False
BMI            False
Pedigree       False
Age            False
Outcome        False
dtype: bool
       Pregnancies     Glucose          BP        Skin     Insulin  \
count   768.000000  768.000000  768.000000  768.000000  768.000000   
mean      3.845052  120.894531   69.105469   20.536458   79.799479   
std       3.369578   31.972618   19.355807   15.952218  115.244002   
min       0.000000    0.000000    0.000000    0.000000    0.000000   
25%       1.000000   99.000000   62.000000    0.000000    0.000000   
50%       3.000000  117.000000   72.000000   23.000000   30.500000   
75%       6.000000  140.250000   80.000000   32.000000  127.250000   
max      17.000000  199.000000  122.000000   99.000000  846.000000   

  

In [51]:
X=dia.loc[:,dia.columns!="Outcome"]
Y=dia.loc[:,dia.columns=="Outcome"]

X=scaling(X,"stand")

X=scaling(X,"norm")

train_X,test_X,train_Y,test_Y=train_test_split(X,Y,test_size=0.2)

In [53]:
from sklearn import linear_model,neighbors,svm,naive_bayes

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import VotingClassifier

In [55]:

def hard_voting(train_X,train_Y,test_X,test_Y):

    m1=neighbors.KNeighborsClassifier(n_neighbors=5)
    m2=linear_model.Perceptron(tol=1e-2,random_state=0)
    m3=svm.SVC(gamma=0.001)
    
    ######## Hard voting ###############
    
    
    
    vote_class=VotingClassifier([("KNN",m1),("Prc",m2),("SVM",m3)])
    
    vote_class.fit(train_X,train_Y)
    
    hard_test_pred=vote_class.predict(test_X)
    
    print("Accuracy of hard voting is :",accuracy_score(hard_test_pred,test_Y))
    print(confusion_matrix(hard_test_pred,test_Y))
    
    hard_train_pred=vote_class.predict(train_X)
    
    print("Accuracy of train data is: ",accuracy_score(hard_train_pred,train_Y))
    print(confusion_matrix(hard_train_pred,train_Y))

In [57]:
############################### soft voting  #######################
def soft_voting(train_X,train_Y,test_X,test_Y):    
    m4=neighbors.KNeighborsClassifier(n_neighbors=5)
    m5=naive_bayes.GaussianNB()
    m6=svm.SVC(gamma=0.001,probability=True)
    
    voting2=VotingClassifier([("KNN",m4),("NB",m5),("SVM",m6)],voting="soft")
    
    voting2.fit(train_X,train_Y)
    
    hard_test_pred2=voting2.predict(test_X)
    
    print("Accuracy of soft voting test is :",accuracy_score(hard_test_pred2
                                                             ,test_Y))
    print(confusion_matrix(hard_test_pred2,test_Y))
    hard_train_pred2=voting2.predict(train_X)
    
    print("Accuracy of soft voting train data is: ",
          accuracy_score(hard_train_pred2,train_Y))
    print(confusion_matrix(hard_train_pred2,train_Y))

In [59]:
print("\n")
hard_voting(train_X,train_Y["Outcome"],test_X,test_Y["Outcome"])

soft_voting(train_X,train_Y["Outcome"],test_X,test_Y["Outcome"])
print("\n")



Accuracy of hard voting is : 0.7662337662337663
[[102  30]
 [  6  16]]
Accuracy of train data is:  0.7687296416938111
[[376 126]
 [ 16  96]]
Accuracy of soft voting test is : 0.7857142857142857
[[91 16]
 [17 30]]
Accuracy of soft voting train data is:  0.7882736156351792
[[340  78]
 [ 52 144]]




In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [61]:
estimators = [
    ('dt', DecisionTreeClassifier()),
    ('svc', SVC(probability=True))
]

# Stacking model
stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    stack_method='predict_proba'
)

# Train
stack_model.fit(train_X, train_Y)

# Predict
y_pred = stack_model.predict(test_X)

print("Stacking Accuracy:", accuracy_score(test_Y, y_pred))

C:\Users\eagle\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:103: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\eagle\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:139: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Stacking Accuracy: 0.8051948051948052


In [37]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


# Load dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Base models
estimators = [
    ('dt', DecisionTreeClassifier()),
    ('svc', SVC(probability=True))
]

# Stacking model
stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    stack_method='predict_proba'
)

# Train
stack_model.fit(X_train, y_train)

# Predict
y_pred = stack_model.predict(X_test)

print("Stacking Accuracy:", accuracy_score(y_test, y_pred))



Stacking Accuracy: 0.9649122807017544
